In [1]:
import os
import glob

os.environ["CONFIG_APP_DIR"] = "tests/test_custom/config_app/"

In [2]:
import sys
import pickle
os.chdir("..")
from omegaconf import OmegaConf
from etl_pipeline.config import pipeline_config
from etl_pipeline.custom.ms.payload_loader import PayloadLoader

import json
from etl_pipeline.data_processor_engine.json_engine.json_engine import JsonProcessingEngine

In [3]:
from etl_pipeline.config import columns_namespace as cn

In [4]:
ETL_SAMPLES_PATH = '/v/region/na/appl/itlnc/s8/data/uat/silent8/name-screening/etl_samples/complete_info/'

In [5]:
DATASET_NAME = 'WM' # 'ISG_Daily' # 'WM' # 'ISG_Weekly 
DATASET_TYPE =  'Address' # 'Address' # Party # Account
DATASET = DATASET_NAME + '_' + DATASET_TYPE

In [6]:
complete_info = glob.glob(ETL_SAMPLES_PATH + DATASET + '/*.json')

### Testing flow

In [16]:
from pipelines.ms.wm_address_pipeline import MSPipeline

engine = JsonProcessingEngine(pipeline_config)
pipeline = MSPipeline(engine, config=pipeline_config)

In [28]:
with open(f'notebooks/sample/wm_party_in_payload_format.json', 'r') as file:
    payload = json.loads(file.read())

payload_json = {key: payload[key] for key in sorted(payload)}
payload_json = PayloadLoader().load_payload_from_json(payload_json)


payload_json['match_ids'] = [i for i in range(len(payload_json["watchlistParty"][cn.MATCH_RECORDS]))]


In [29]:
payload = pipeline.transform_standardized_to_cleansed(payload_json)
new_payloads = pipeline.transform_cleansed_to_application(payload)

2022-03-24 14:24:00,488 — ETL Pipeline — ERROR — Field {'ap': ['ALL_PARTY_DOBS'], 'wl': ['matchRecords.WL_DOB']} does not exist in payload
2022-03-24 14:24:00,489 — ETL Pipeline — ERROR — Field {'ap': ['ALL_TAX_IDS', 'PARTY1_GOVTID1_NUMBER', 'PARTY1_GOVTID2_NUMBER'], 'wl': ['matchRecords.WL_DOCUMENT_NUMBER']} does not exist in payload
2022-03-24 14:24:00,489 — ETL Pipeline — ERROR — Field {'ap': ['ALL_PARTY_EMPLOYERS'], 'wl': ['matchRecords.WL_NAME', 'matchRecords.WL_ALIASES']} does not exist in payload
2022-03-24 14:24:00,489 — ETL Pipeline — ERROR — Field {'ap': ['ALL_PARTY_NAMES'], 'wl': ['matchRecords.WL_NAME', 'matchRecords.WL_ALIASES']} does not exist in payload
2022-03-24 14:24:00,490 — ETL Pipeline — ERROR — Field {'ap': ['ALL_PARTY_TYPES'], 'wl': ['matchRecords.WLP_TYPE']} does not exist in payload
2022-03-24 14:24:00,490 — ETL Pipeline — ERROR — Field {'ap': ['ALL_PARTY_BIRTH_COUNTRIES', 'ALL_PARTY_COUNTRY'], 'wl': ['matchRecords.WL_COUNTRY', 'matchRecords.WL_COUNTRYNAME', 'm

KeyError: 'WL_DOB'

In [22]:
import pandas as pd

In [25]:
out_payload = pd.DataFrame([match for payload in new_payloads   for match in payload['watchlistParty']['matchRecords']])

In [26]:
out_payload[[i for i in out_payload.columns  if i.endswith("_aggregated")]]

,ap_all_dobs_aggregated,wl_all_dobs_aggregated,ap_all_nationalities_aggregated,wl_all_nationalities_aggregated,ap_all_residencies_aggregated,wl_all_residencies_aggregated
0,"[02/31/1900, 04/31/1910]","[MAY 6, 1981, //]",[],[US],[UNITED STATES OF AMERICA],[US]
1,"[02/31/1900, 04/31/1910]","[MAY 6, 1981, //]",[],"[PL, Poland]",[UNITED STATES OF AMERICA],[PL]


# check all test jsons - You need to customize it - please copy paste code from the above class definition

In [ ]:
from tqdm import tqdm
for info in tqdm(complete_info):
    # print('\n' + os.path.split(info)[1])
    with open(info, 'r') as file:
        payload = json.load(file)
    
    
    
    
            
    '''for payload in new_payloads:
        # print('p')
        for match in payload['alert']['matchRecords']:
            # print('match')
            # print("++++++++")
            # print(match["ap_all_dobs_aggregated"], match["wl_all_dobs_aggregated"])
            try:
                ap = match["ap_all_residencies_aggregated"]
                wl = match["wl_all_residencies_aggregated"]
            except:
                break'''
